In [ ]:
#Read train and depth.csv and put them in one dataframe df_Train
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import time
import scipy.signal as sg
import tensorflow as tf
from keras.models import Model
from keras import layers
from keras import backend as K
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


tic = time.time()

num_train_images= 4000
Height = 101
Width = 101

print(os.listdir("../input"))
df_train = pd.read_csv("../input/train.csv")
print("\n Train files shape is ", df_train.shape)

df_depths = pd.read_csv("../input/depths.csv")
df_depths['z'] = df_depths['z'] / np.max(df_depths['z'])
print("\nDepths files shape is ", df_depths.shape)

df_train = df_train.join(df_depths, lsuffix='idl', rsuffix='idr', how ='inner')
df_train.pop('ididr')
df_train.columns=['id', 'rle_mask','depth']
print(df_train.head())                      


In [ ]:
#2. Read train & mask images and add them up df_train 
df_train['images'] = [np.array(Image.open("../input/train/images/{}.png".format(idx)).convert('L'))
                      for idx in df_train['id']]
print("Sample Image Shape is ", df_train['images'][0].shape)
print("No. of train images are ", len(df_train))
print("Sample pixel value of trian image before normalization is  ", df_train['images'][15][10,0])

df_train['images']=df_train['images'] / 255.
print("After normalization,  pixel value is ", df_train['images'][15][10,0])

df_train['masks'] = [np.array(Image.open("../input/train/masks/{}.png".format(idx)).convert('L')) for 
                               idx in df_train['id']]

print("\nSample Mask Shape is ", df_train['masks'][0].shape)
print("Sample pixel value before normalization of mask is  ", df_train['masks'][15][10,0])
print("No. of mask images are ", len(df_train))
df_train['masks']=df_train['masks']/255.
print("Sample pixel value after normalization of mask is  ", df_train['masks'][15][10,0])
print("train df columns are ", df_train.columns)


In [ ]:

#df_blank_masks = df_train[df_train['rle_mask'].isna()]
#limit = 4
#fig = plt.figure()
#for i in range(1,limit+1):
    #img = np.random.randint(10, size=(h,w))
  #  plt.subplot(limit,2,i*2-1)#2,1)
  #  plt.imshow(df_blank_masks['images'][6])
  #  plt.subplot(limit,2,i*2)#2,1)
  #  plt.imshow(df_blank_masks['masks'][6])
#plt.show()
    

In [ ]:
train_x = np.array(df_train['images'])
train_x = np.concatenate(train_x,axis=0)
train_x = np.reshape(train_x,(4000,101,101,1))
print("Train Shape = ",train_x.shape)
print("Sample Pixel VAlue", train_x[0,1,0])
#print(df_train['masks'][5][0])

train_y = np.array(df_train['masks'])
train_y = np.concatenate(train_y,axis=0)
train_y = np.reshape(train_y,(4000,101,101,1))
#train_y = train_y / train_y.max()
print("Mask Shape = ",train_y.shape)
train_y = np.round(train_y)
print("Sample Pixel VAlue", train_y[15,10,0])

In [ ]:
df_train['empty_mask'] = 1.5
df_train['empty_mask']= df_train['rle_mask'].where(df_train['rle_mask'].isnull(),0)
df_train['empty_mask'] = df_train['empty_mask'].fillna(1.)
df_train['empty_mask'].head(15)
train_emptymask_y = np.array(df_train['empty_mask'])
train_emptymask_y = train_emptymask_y.reshape(4000,1)
print(train_emptymask_y.shape)
#print(train_emptymask_y[0:8])

In [ ]:
import random as rn
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1234)
rn.seed(1234)


session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

#print(depth_np.shape)
#depth_np = np.zeros(shape = (num_train_images,12,12,1))
#for i in range(num_train_images):
#    depth_np[i] = np.full(shape=(12,12,1), fill_value=df_train['depth'].loc[i])  

#df_train['depth_image'] = pd.Series(map(lambda x:np.full(shape=(50,50,1), 
#                                                         fill_value=x), df_train['depth']))

#depth_np = np.array(df_train['depth_image'])
#depth_np = np.concatenate(depth_np,axis=0)
#depth_np = np.reshape(depth_np,(num_train_images,50,50,1))

depth_np_new = np.array(df_train['depth'])
print("depth", depth_np_new[0:5])
print("depth shape ", depth_np_new.shape)
depth_np_new = depth_np_new.reshape((num_train_images,1))
#depth_np = np.concatenate((depth_np, depth_np,depth_np, depth_np))
print("depth shape ", depth_np_new.shape)
#print(a.shape)
#depth_np= depth_np/depth_np.max()
#print(a[2][2])
print(df_train.columns)


In [ ]:
Activation1 = 'relu'
Activation2 = 'tanh'

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
    
    if activation == True:
        x = layers.BatchNormalization()(x)
        x = layers.Activation(Activation1)(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    print("Applied residual block on size")
    #x = layers.Activation(Activation1)(blockInput)
    x = convolution_block(blockInput, num_filters, (3,3), activation=True )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = layers.Add()([x, blockInput])
    if batch_activate:
        x = layers.BatchNormalization()(x)
        x = layers.Activation(Activation1)(x)
    return x

In [ ]:
img_input = layers.Input(shape=(Height, Width,1), name = 'img_input')
depth_input_new = layers.Input(shape=(1,), name='depth_input_new')

print("101 -> 100")
x = layers.Conv2D(filters = 16, kernel_size= (2,2), padding='valid', activation=Activation1)(img_input)
print(x)
x = layers.BatchNormalization()(x)
      
print("100 -> 50")
x = layers.MaxPooling2D(pool_size=(2,2))(x)
      
x = layers.Conv2D(filters = 64, kernel_size= (2,2), padding='same', activation=Activation1)(x)
print(x)
x = layers.BatchNormalization()(x)
      
print("50 -> 25")
x = layers.MaxPooling2D(pool_size=(2,2))(x)
      
x = residual_block(x, num_filters = 64, batch_activate = True)
x = layers.Dropout(0.25)(x)
      
print("25->24")
x = layers.Conv2D(filters = 128, kernel_size= (2,2), padding='valid', activation=Activation1)(x)
print(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.25)(x)

print("24 -> 12")
x = layers.MaxPooling2D(pool_size=(2,2))(x)
print(x)
x = residual_block(x, num_filters = 128)
print(x)
x = layers.Dropout(0.25)(x)

#x = layers.Concatenate()([x,depth_input_new])

x = layers.Flatten()(x)

print(x)

# Concatenate Depth
#depth_x = layers.Dense(1)(depth_input_new)#(, activation=false)
#depth_x = layers.Flatten()(depth_x)

x= layers.Dense(256,activation = Activation1)(x)
x = layers.Dropout(0.2)(x)
x = layers.Concatenate()([x, depth_input_new])
print(x)

x= layers.Dense(128,activation = Activation1)(x)

print(x)

x= layers.Dense(64,activation = Activation1)(x)
print(x)

output = layers.Dense(1,activation = 'sigmoid')(x)
print(output)

bmodel = Model([img_input,depth_input_new], output)
bmodel.summary()

In [ ]:
Batch_size = 128
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True)

def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=Batch_size,seed = 1234, shuffle=True)
    genX2 = gen.flow(X1,X2, batch_size=Batch_size,seed = 1234, shuffle=True)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]


def gen_flow_for_two_inputs2(X, I, Y):
     # suffled indices    
    idx = np.random.permutation(X.shape[0])
    print("Shape[0] ",X.shape[0])
    print("permutation ",idx)
    print("length ",len(idx))
    idx0 = 0
    for batch in gen.flow( X[idx], Y[idx], batch_size=Batch_size, shuffle=False):
        idx1 = idx0 + batch[0].shape[0]
        print("Before ", idx0, " ", idx1)
        yield [batch[0], I[ idx[ idx0:idx1 ] ]], batch[1]
        idx0 = idx1
        if idx1 >= X.shape[0]:
            break
        print("After ", idx0, " ", idx1)


        

In [ ]:
df_test = df_depths[~df_depths['id'].isin(df_train['id'])]

print("Length of df_test is ", len(df_test))
df_test = df_test.reset_index()
df_test.pop('index')
print(df_test.head())

In [ ]:
df_test['images'] = [np.array(Image.open("../input/test/images/{}.png".format(idx)).convert('L')) for 
                               idx in df_test['id']]

print(df_test.head())
print("Sample Image Shape is ", df_test['images'][10].shape)

In [ ]:

#df_test['images'] = pd.Series(map(lambda x: np.delete(x,np.s_[1:],2), df_test['images']))
#print("After optimization,  Image Shape is ", df_test['images'][0].shape)
print("No. of test images are ", len(df_test))
print("Before normalization,  pixel value is ", df_test['images'][10][2,0])
df_test['images']=df_test['images'] / 255
print("After normalization,  pixel value is ", df_test['images'][10][2,0])

print(df_test.columns)

test_x = np.array(df_test['images'])
test_x = np.concatenate(test_x,axis=0)
test_x = np.reshape(test_x,(18000,101,101,1))
print("Test Shape = ",test_x.shape)
print("Sample Pixel VAlue", test_x[10,2,0])

In [ ]:

#df_test['depth_image'] = pd.Series(map(lambda x:np.full(shape=(50,50,1), 
#                                                         fill_value=x), df_test['z']))

#depth_test_np = np.array(df_test['depth_image'])
#depth_test_np = np.concatenate(depth_test_np,axis=0)
#depth_test_np = np.reshape(depth_test_np,(18000,50,50,1))
depth_test_np_new = np.array(df_test['z'])

#print(a.shape)
#depth_np= depth_np/depth_np.max()
#print(a[2][2])
print(df_test.columns)
depth_test_np_new = np.reshape(depth_test_np_new,(18000,1))
print(depth_test_np_new.shape)

In [ ]:
def recall1(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    #true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    #true_positives = K.sum(y_true * y_pred)
    true_positives = np.sum(y_true * y_pred)
    #possible_positives = K.sum(y_true)
    possible_positives = np.sum(y_true)
    recall = true_positives / (possible_positives + 0.00001)
    return recall

def precision1(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    #true_positives = K.sum(y_true * y_pred)
    #predicted_positives = K.sum(y_pred)
    true_positives = np.sum(y_true * y_pred)
    predicted_positives = np.sum(y_pred)
    precision = true_positives / (predicted_positives + 0.00001)
    return precision
    
def f1(y_true, y_pred):
    precision = precision1(y_true, y_pred)
    recall = recall1(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+0.00001))

In [ ]:
epochs=5
Batch_size = 64
x_train, x_val, depth_train, depth_val,  y_train, y_val = train_test_split(train_x, depth_np_new,
                                                train_emptymask_y, test_size=0.25, random_state=42)

reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, 
                              min_lr=0.0001, verbose=1)

bmodel = Model([img_input,depth_input_new], output)
    #bmodel = Model(img_input, output)
bmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gen_flow = gen_flow_for_two_inputs(x_train, depth_train, y_train)

bmodel.fit_generator(gen_flow,validation_data= ([x_val, depth_val],
                                                      y_val), 
                         steps_per_epoch=len(x_train) / Batch_size, 
                         epochs=epochs, callbacks = [reduce_lr], verbose = 1 )

In [ ]:
pred_y_val = bmodel.predict([x_val, depth_val])

In [ ]:
#val_pred_y = []#np.zeros(shape=(n_splits, int(num_train_images/n_splits)+1, 1))
#y_test_pred = np.zeros(shape=(n_splits, 18000,1))
#train_y = np.append(train_y, [np.fliplr(x) for x in train_y], axis=0)
#y_test_pred_lr = np.append(y_test_pred_lr, [np.fliplr(x) for x in y_test_pred_lr], axis=0)
#y_test_pred_up = [np.fliplr(x) for x in y_test_pred_lr], axis=0)
#y_test_pred_lr = np.zeros(shape=(n_splits, 18000,1))
#y_test_pred_up = np.zeros(shape=(n_splits, 18000,1))


In [ ]:
n_splits=5
epochs=50
base_name = 'binary_kfold'
#x_train, x_val, y_train, y_val = train_test_split(train_x, 
 #                                               train_emptymask_y, test_size=0.33, random_state=42)
#early_stopping = EarlyStopping(monitor='acc', patience=4)
#bmodel.fit(x_train,y_train,validation_data=[x_val, y_val], epochs=50,verbose=1, 
#          batch_size=96, callbacks = [early_stopping])

kf = StratifiedKFold(n_splits=n_splits,random_state=1234, shuffle=True)

pred=[]
models = []
val_indexes=[]

val_pred_y = []#np.zeros(shape=(n_splits, int(num_train_images/n_splits)+1, 1))
y_test_pred = np.zeros(shape=(n_splits, 18000,1))
y_test_pred_lr = np.zeros(shape=(n_splits, 18000,1))
y_test_pred_up = np.zeros(shape=(n_splits, 18000,1))

# Finally create generator
i=0
for train_index, val_index in kf.split(train_x, train_emptymask_y):
    print(i)
    model_name = base_name + str(i) + 'h5'
    #model_checkpoint = ModelCheckpoint(model_name, monitor='acc', 
    #                               mode = 'max', save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, 
                              min_lr=0.0001, verbose=1)

    bmodel = Model([img_input,depth_input_new], output)
    #bmodel = Model(img_input, output)
    bmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    gen_flow = gen_flow_for_two_inputs(train_x[train_index], depth_np_new[train_index]
                                       ,train_emptymask_y[train_index])
    
    #bmodel.fit_generator(x = train_x[train_index] ,y=train_emptymask_y[train_index],
    #           batch_size=128,epochs=epochs, verbose = 1,callbacks = [model_checkpoint,reduce_lr],
    #          validation_data = [train_x[val_index],train_emptymask_y[val_index]])
               
    bmodel.fit_generator(gen_flow,validation_data= ([train_x[val_index], depth_np_new[val_index]],
                                                      train_emptymask_y[val_index]), 
                         steps_per_epoch=len(train_x[train_index]) / Batch_size, 
                         epochs=epochs, callbacks = [reduce_lr], verbose = 1 )
    
    #Predict on Cross Validation Set:
    val_pred_y.append(bmodel.predict([train_x[val_index],  depth_np_new[val_index]]))
    val_indexes.append(val_index)
    
    #Predict on Test  Images Set:
    #print(y_test_pred.shape)
    #print(test_x.shape)
    #print(depth_test_np_new.shape)
    y_test_pred[i,:,:] =  bmodel.predict([test_x, depth_test_np_new])
    y_test_pred_lr[i,:,:] =  bmodel.predict([[np.fliplr(x) for x in test_x],depth_test_np_new])
    #y_test_pred_up[i,:,:] =  bmodel.predict([np.flipud(test_x),depth_test_np_new])
        
    pred.append(bmodel.evaluate([train_x[val_index], depth_np_new[val_index]], 
                                train_emptymask_y[val_index]))
    models.append(bmodel)
    i+=1

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(16, 8))
columns = 1
rows = 4
%matplotlib inline
fig, axs = plt.subplots(rows, columns)
index = np.random.randint(0,len(x_val),rows)
#print(y_pred[0,:,:,0].shape)
#index = [206, 831, 524, 676]
val_x_pred_new = np.round(pred_y_val-0.25)#-0.05)
#val_x_pred_new = np.round(val_x_pred-0.25)#-0.05)
print(index)
#plt.imshow(y_pred[100,:,:,0])
#for i in range(4):
    #img = np.random.randint(10, size=(h,w))
    #fig.add_subplot(rows, columns, i)
        
    #axs[i, 0].imshow(x_val[index[i],:,:,0])#, cmap=cmap)
    #print("Actual Image Classification at {} is {}".format(index[i], y_val[index[i]]))
    #axs[i,1].imshow(y_val[index[i],:,:,0])#, cmap=cmap)
    #print("Image Prediction at {} is {}".format(index[i], val_x_pred_new[index[i]]))
    #axs[i,2].imshow(val_x_pred_new[index[i],:,:,0])#, cmap=cmap)
    #plt.imshow(y_pred_new[x[0],:,:,0])
#plt.show()

In [ ]:
kf.get_n_splits(train_x, train_emptymask_y)

In [ ]:
print(pred)
print("Mean Val Loss and Accuracy are ", np.mean(pred, axis=0))
print("Last Val Loss and Accuracy are ", pred[-1])

In [ ]:
print(len(models))
#print(train_pred.shape)
for i in range(n_splits):
    print(np.mean(val_pred_y[i]))
    print(val_pred_y[i].shape)
    

#val_mean = np.mean(val_pred, axis =0)
#val_pred_y_mean = np.mean(val_pred_y, axis =0)
#print(val_mean.shape)
#print(val_pred_y_mean.shape)
#val_pred_y = np.array(val_pred_y)
#print(val_pred_y.mean())

In [ ]:
"""i =0
p=[]
f=[]
r=[]
pf=[]
rf=[]
ff=[]

for z in np.arange(0.25,0.8,0.05):
    p.append(precision1(train_emptymask_y[val_indexes[0]],np.round(val_pred_y[0] - (z-0.5))))
    r.append(recall1(train_emptymask_y[val_indexes[0]],np.round(val_pred_y[0] - (z-0.5))))
    f.append(f1(train_emptymask_y[val_indexes[0]],np.round(val_pred_y[0] - (z-0.5))))
    
    pf.append(precision1(train_emptymask_y[val_indexes[-1]],np.round(val_pred_y[-1] - (z-0.5))))
    rf.append(recall1(train_emptymask_y[val_indexes[-1]],np.round(val_pred_y[-1] - (z-0.5))))
    ff.append(f1(train_emptymask_y[val_indexes[-1]],np.round(val_pred_y[-1] - (z-0.5))))
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(20, 10))


ax1.plot(np.arange(0.25,0.8,0.05),p,'blue',  label='Mean Precision')
ax1.plot(np.arange(0.25,0.8,0.05),r,'red',  label='Mean Recall')
ax1.plot(np.arange(0.25,0.8,0.05),f,'black',  label='Mean F1')
#plt.plot(np.arange(0.25,0.8,0.05))


ax2.plot(np.arange(0.25,0.8,0.05),pf,'blue',  label='Final Precision')
ax2.plot(np.arange(0.25,0.8,0.05),rf,'red',  label='Final Recall')
ax2.plot(np.arange(0.25,0.8,0.05),ff,'black',  label='Final F1')
#plt.ylabel('S')
ax1.grid(True)
ax1.legend()
#plt.title("First Model Prediction")
ax2.grid(True)
ax2.legend()
#plt.xtitle("Last Model Prediction")
plt.xlabel("Threshold")
#plt.xlabel('Last Model')
plt.show()   
""""

In [ ]:
print("First test predicdtion shape without modification ", y_test_pred.shape)
y_test_pred_mean = np.mean([y_test_pred, y_test_pred_lr], axis=0)
print("sum of all predicdtions ", y_test_pred_mean.sum())
print("\n sum of first prediction \n")
print(y_test_pred[0].sum())
print(y_test_pred_lr[0].sum())
#print(y_test_pred_up[0].sum())

print("\n sum of last prediction \n")
print(y_test_pred[-1].sum())
print(y_test_pred_lr[-1].sum())
#print(y_test_pred_up[-1].sum())

y_test_pred_mean_again = np.mean(y_test_pred_mean, axis=0)
print("Mean of all predictions shape ", y_test_pred_mean_again.shape)


In [ ]:
y_test_pred_mean_df = pd.DataFrame(data = {'id':df_test['id'], 'mask':y_test_pred_mean_again[:,0]})
y_test_pred_mean_df.to_csv("binary_output_mean_test.csv", index=False)
print(y_test_pred_mean_df.head())
print("Test mean prediction written")
#print(y_test_pred_mean_again[:,0].shape)

y_test_pred_last_df = pd.DataFrame(data = {'id':df_test['id'], 'mask':list(y_test_pred_mean[-1,:,:])})
print(y_test_pred_last_df.head())
y_test_pred_last_df.to_csv("binary_output_last_test.csv", index=False)
print("Test last model prediction written")

#true_y_df = pd.DataFrame(data = {'id':df_test['id'], 'mask':train_cleanmask_y})
#true_y_df.to_csv("true_y.csv", index=False)
#df_train[['id','empty_mask']].to_csv("true_y_train.csv", index=False)


In [ ]:
tac = time.time()
print(tac-tic)

In [ ]:
#output = pd.DataFrame( data = {'id': df_test['id'], 'rle_mask' : new_y_pred} )
#output.to_csv("output_6.csv", index=False)
